In [1]:
# This file is used to handle with the origianl collected data(json file), 
#   including calculate the duration between each action, transform the text of each action to a vector
#   so that it can create a new json file, which contains features that need to be visualized
import json
from sentence_transformers import SentenceTransformer
from datetime import datetime
import pandas as pd

In [2]:
# function used to split the type feature into 2 demensions 0&1
def Type_Judge(type, instance):
    if(instance == type):
        return 1
    else:
        return 0

In [3]:
def transform(filename):
    
    # Load the json file and the model
    with open('example' + filename + '.json') as f:
        data = json.load(f)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    index = 0

    actionList = [] 
    while(index < len(data)):
        time = datetime.fromisoformat(data[index]['time'].replace('Z', '+00:00'))
        # Create a json object to store an action's information
        action = {
            "id" : index,
            "duration": 0,
            "time": data[index]['time'],
            "search": Type_Judge('search', data[index]['type']),
            "link": Type_Judge('link', data[index]['type']),
            "revisit":Type_Judge('revisit', data[index]['type']),
            "highlight":Type_Judge('highlight', data[index]['type']),
            "filter":Type_Judge('filter', data[index]['type']),
            "zoomLevel": data[index]['zoomLevel'],
            "title": data[index]['text'],
            "text": model.encode(data[index]['text']).tolist(),
        }
        # Add action to a list
        actionList.append(action)
        index = index + 1

    # Calculate the duraction between each action
    i = 0
    while(i < (len(actionList)-1)):
        time1 = datetime.fromisoformat(actionList[i+1]['time'].replace('Z', '+00:00'))
        time2 = datetime.fromisoformat(actionList[i]['time'].replace('Z', '+00:00'))
        actionList[i]['duration'] =  (time1 - time2).seconds
        i = i + 1

    # Write the action list to a new json file
    with open('transformed' + filename + '.json', 'w') as f:
        json.dump(actionList, f)



In [4]:
transform('01')
transform('02')
transform('03')
transform('04')
transform('05')
transform('06')